In [ ]:

import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import json
import os

# 기본 레이아웃
layout_full = widgets.Layout(width='100%')
layout_half = widgets.Layout(width='48%')
layout_third = widgets.Layout(width='32%')
layout_button = widgets.Layout(width='100%', height='36px')
layout_progress = widgets.Layout(width='100%', height='20px')

todo_items = []


In [ ]:

def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_label.value = "최종 진행률"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    progress_bar.value = avg
    progress_label.value = f"최종 진행률"
    progress_text.value = f"{int(avg)}%"


In [ ]:

def refresh_list():
    list_box.children = []
    for idx, item in enumerate(todo_items, start=1):
        title = widgets.Label(value=f"# {idx}. {item['title']}", layout=layout_full)

        bar_color = f"rgb({255 - int(item['progress']*2.55)}, {255 - int(item['progress']*2.55)}, {255 - int(item['progress']*2.55)})"
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100,
                                    layout=layout_progress, style={'bar_color': bar_color})
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=1, layout=layout_full)

        percent = widgets.Label(value=f"{item['progress']}%", layout=layout_full)

        due = widgets.Label(value=f"마감일  {item['due']}", layout=layout_full)
        priority = widgets.Label(value=f"중요도  {item['priority']}", layout=layout_full)

        complete_btn = widgets.Button(description='완료✅', layout=layout_half, button_style='success')
        delete_btn = widgets.Button(description='삭제❌', layout=layout_half, button_style='danger')

        def on_slider_change(change, i=item, p=percent, b=bar):
            i['progress'] = change['new']
            p.value = f"{change['new']}%"
            grey = 255 - int(change['new'] * 2.55)
            b.style.bar_color = f"rgb({grey},{grey},{grey})"
            b.value = change['new']
            update_overall_progress()

        def on_complete_click(btn, i=item):
            i['progress'] = 100
            refresh_list()
            update_overall_progress()

        def on_delete_click(btn, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()

        slider.observe(on_slider_change, names='value')
        complete_btn.on_click(on_complete_click)
        delete_btn.on_click(on_delete_click)

        box = widgets.VBox([title, bar, slider, percent, due, priority,
                            widgets.HBox([complete_btn, delete_btn], layout=layout_full)],
                           layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px'))
        list_box.children += (box,)


In [ ]:

title_input = widgets.Text(placeholder='할 일 입력', layout=layout_full)
progress_input = widgets.IntSlider(value=0, min=0, max=100, layout=layout_full)
due_input = widgets.DatePicker(layout=layout_full)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_full)

add_btn = widgets.Button(description='추가', layout=layout_full, button_style='success')
save_btn = widgets.Button(description='저장', layout=layout_half, button_style='info')
load_btn = widgets.Button(description='불러오기', layout=layout_half, button_style='warning')

progress_bar = widgets.FloatProgress(min=0, max=100, layout=layout_full, style={'bar_color': 'black'})
progress_text = widgets.Label(value="0%", layout=widgets.Layout(width='auto', margin='0 0 0 10px'))
progress_label = widgets.Label(value="최종 진행률", layout=layout_full)

list_box = widgets.VBox()

def add_task(btn):
    if not title_input.value.strip():
        return
    todo_items.append({
        'title': title_input.value.strip(),
        'progress': progress_input.value,
        'due': due_input.value.strftime('%Y-%m-%d') if due_input.value else '',
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    refresh_list()
    update_overall_progress()

def save_data(btn):
    with open('todo.json', 'w') as f:
        json.dump(todo_items, f)

def load_data(btn):
    if not os.path.exists('todo.json'):
        return
    with open('todo.json', 'r') as f:
        data = json.load(f)
        todo_items.clear()
        todo_items.extend(data)
    refresh_list()
    update_overall_progress()

add_btn.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

display(widgets.VBox([
    widgets.HTML(value="<h3 style='color:white;'>📂 DAVID mk38 - 짙은 배경 버전</h3>"),
    title_input,
    progress_input,
    due_input,
    priority_input,
    add_btn,
    widgets.HBox([save_btn, load_btn]),
    widgets.HTML(value="<b style='color:white;'>최종 진행률</b>"),
    widgets.HBox([progress_bar, progress_text]),
    list_box
], layout=widgets.Layout(background_color='#1e1e1e', padding='10px')))
refresh_list()
update_overall_progress()
